# 予測精度の最も良いランダムフォレストの予測結果を、Googleスプレッドシートに出力する

In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from sklearn.metrics import confusion_matrix, accuracy_score,precision_score,recall_score, f1_score,classification_report
from random import randint, shuffle
import os
import seaborn as sns




In [2]:
# 予測した結果を読み込む
randomforest_result = pd.read_csv('Desktop/インテグ課題/NASA/submission3.csv')

In [3]:
randomforest_result.head()

,engine_no,result
0,0,1
1,1,1
2,2,0
3,3,0
4,4,1


In [4]:
#result=1のみ取り出す
google_sp = randomforest_result[randomforest_result['result'] == 1]

In [5]:
google_sp.head()

,engine_no,result
0,0,1
1,1,1
4,4,1
7,7,1
8,8,1


In [6]:
google_sp.shape[0]

342

In [135]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
 
scope = ['スプレッドシートのURL']
 
# ダウンロードしたjsonファイル名を記入
json_file = 'ダウンロードしたjsonファイル名'
# スプレッドシートIDを記入
sheet_id = 'スプレッドシートID'
# スプレッドシートのシート名を記入
sheet_name = 'スプレッドシートのシート名'
 
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_file, scope)
gc = gspread.authorize(credentials)
sp = gc.open_by_key(sheet_id)
wks = sp.worksheet(sheet_name)
 


In [136]:
#型を確認
print(google_sp.dtypes)

engine_no    int64
result       int64
dtype: object


In [137]:
#jsonに出力できるタイプに変更
google_sp=google_sp.astype(({"engine_no": object}))
google_sp=google_sp.astype(({"result": object}))

In [138]:
import pandas as pd
import gspread
import json
from oauth2client.service_account import ServiceAccountCredentials 

# 2つのAPIを記述しないとリフレッシュトークンを3600秒毎に発行し続けなければならない
scope = ['API','API']

# 認証情報設定
# ダウンロードしたjsonファイル名をクレデンシャル変数に設定（秘密鍵、Pythonファイルから読み込みしやすい位置に置く）
credentials = ServiceAccountCredentials.from_json_keyfile_name('秘密鍵のJSON', scope)

# OAuth2の資格情報を使用してGoogle APIにログインします。
gc = gspread.authorize(credentials)

#共有設定したスプレッドシートキーを変数[SPREADSHEET_KEY]に格納する。
SPREADSHEET_KEY = 'スプレッドシートキー'

#共有設定したスプレッドシートのワークシート1を開く
worksheet = gc.open_by_key(SPREADSHEET_KEY).sheet1

# 行と列が可変長のDataFrameを書き出す

In [140]:
# 数字からアルファベットを返す関数
# 例：26→Z、27→AA、10000→NTP
def toAlpha(num):
    if num<=26:
        return chr(64+num)
    elif num%26==0:
        return toAlpha(num//26-1)+chr(90)
    else:
        return toAlpha(num//26)+chr(64+num%26)


In [141]:
# ヘッダーと中身をスプレッドシートに書き出す
cell_list = worksheet.range('A1:'+toAlpha(col_lastnum)+str(row_lastnum+1))
for cell in cell_list:
    if cell.row == 1:
        val = google_sp.columns[cell.col-1]
    else:
        val = google_sp.iloc[cell.row-2][cell.col-1]
    cell.value = val
worksheet.update_cells(cell_list)

{'spreadsheetId': '1aoWPywCi8tdFMVK2ytNxg00arlJa7WU3yn1IVJcye38',
 'updatedRange': "'予測結果'!A1:B357",
 'updatedRows': 357,
 'updatedColumns': 2,
 'updatedCells': 714}

In [142]:
#参考サイト
#https://tanuhack.com/gspread-dataframe/